In [2]:

from floodligt_functions import clean_metadata
from floodligt_functions import read_position_data_jsonl

import numpy as np
from pathlib import Path
import json
import io
import polars as pl
import floodlight
#from floodlight.io.secondspectrum import read_position_data_jsonl
from floodlight.io.secondspectrum import read_teamsheets_from_meta_json
import json
import tempfile
from typing import Union, Tuple, Dict
from pathlib import Path
from floodlight.core.pitch import Pitch
from floodlight.io.utils import get_and_convert
from floodlight.core.teamsheet import Teamsheet
from floodlight.core.xy import XY
from floodlight.core.code import Code
from floodlight.models.kinematics import AccelerationModel
from floodlight.models.kinematics import DistanceModel
from floodlight.models.kinematics import VelocityModel
from floodlight.models.kinetics import MetabolicPowerModel


In [3]:
filepath_position = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/tracking-produced.jsonl")

In [4]:
filepath_metadata = Path("SecondSpectrum/Second Spectrum/Anderlecht - Club Brugge/rsc-bru_metadata.json")

In [5]:
filepath_cleaned_metadata = Path("cleaned_metadata.json")

In [6]:
clean_metadata(filepath_metadata, filepath_cleaned_metadata)

In [7]:
position_data = read_position_data_jsonl(
    filepath_position,
    filepath_cleaned_metadata
)

In [8]:
xy_objects = position_data[0]

In [ ]:
xy_objects['HT2']['Home']

XY(xy=array([[39.94, -0.09,   nan, ...,   nan,   nan,   nan],
       [39.8 , -0.09,   nan, ...,   nan,   nan,   nan],
       [39.67, -0.09,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
      shape=(2901801, 42)), framerate=25, direction='rl')

: 

In [9]:
np.save("np_arrays/xy.npy", xy_objects['HT2']['Home'])

In [9]:
xy_objects['HT2']['Home'].xy

array([[39.94, -0.09,   nan, ...,   nan,   nan,   nan],
       [39.8 , -0.09,   nan, ...,   nan,   nan,   nan],
       [39.67, -0.09,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan]],
      shape=(2901801, 42))

In [9]:
halfs = ["HT2"]#["HT1", "HT2"]
teams = ["Home", "Away"]

In [11]:
# get list of home players
# this will be used to match player with correct position data
home_players = [player for player in position_data[3]['Home']["player"]]

# get list of away players
away_players = [player for player in position_data[3]['Away']["player"]]

In [12]:
possession_status = {}

for half in halfs:
    half_possession = position_data[1][half].code

    # Convert np.nan to None so Polars can handle them as nulls
    half_possession = [x if isinstance(x, str) else None for x in half_possession]

    # Create DataFrame
    possession_status[half] = pl.DataFrame({
        f"{half} possession": half_possession
    })

In [13]:
ball_status = {}

for half in halfs:
    half_ball_status = position_data[2][half].code
    half_ball_status = [x if isinstance(x, str) else None for x in half_ball_status]
    ball_status[half] = pl.DataFrame({
        f"{half} ball status": half_ball_status
    })


In [12]:
xy_objects = position_data[0]

In [ ]:
# Dictionary to store results
acceleration_dfs = {}

for half in halfs:
    acceleration_dfs[half] = {}
    
    for team in teams:
        am = AccelerationModel()
        am.fit(xy_objects[half][team])
        acceleration = am.acceleration()
        np_acceleration_array = acceleration.property
        acceleration_dfs[half][team] = np_acceleration_array

In [15]:
acceleration_dfs['HT2']['Home']

array([[-3.125     ,         nan, -1.83058262, ...,         nan,
                nan,         nan],
       [-3.125     ,         nan, -0.91529131, ...,         nan,
                nan,         nan],
       [-1.5625    ,         nan, -0.64720869, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]], shape=(2901801, 21))

In [16]:
np.save("np_arrays/acceleration.npy", acceleration_dfs['HT2']['Home'])

In [10]:
velocity_dfs = {}

for half in halfs:
    velocity_dfs[half] = {}
    
    for team in teams:
        vm = VelocityModel()
        vm.fit(xy_objects[half][team])
        velocity = vm.velocity()
        np_velocity_array = velocity.property
        velocity_dfs[half][team] = np_velocity_array

In [ ]:
np.save("np_arrays/velocity.npy", velocity_dfs['HT2']['Home'])

: 